## This file helps to Build and test the LLama model and Langchain functions

In [2]:
# Importing packages

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [3]:
#setting up env
load_dotenv()

True

In [4]:
#Building LLM
llm = ChatGroq(
    model='llama3-8b-8192',
    groq_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.5
    )

In [5]:
#Getting response 
response = llm.invoke("How to book movie tickets in USA?")
print(response)

content="Booking movie tickets in the USA is a relatively straightforward process. Here are the steps:\n\n**Option 1: Online Booking**\n\n1. Visit the website of a movie theater chain or a ticketing website like:\n\t* Fandango (fandango.com)\n\t* Atom Tickets (atomtickets.com)\n\t* MovieTickets (movietickets.com)\n\t* AMC Theatres (amctheatres.com)\n\t* Regal Cinemas (regmovies.com)\n2. Enter your location or zip code to find nearby theaters.\n3. Select the movie you want to watch and the showtime you prefer.\n4. Choose your seats (if available) and add any additional features like 3D or IMAX.\n5. Enter your payment information and confirm your booking.\n6. You'll receive a confirmation email with your ticket details.\n\n**Option 2: Mobile App**\n\n1. Download the mobile app of a movie theater chain or a ticketing app like:\n\t* Fandango (iOS, Android)\n\t* Atom Tickets (iOS, Android)\n\t* MovieTickets (iOS, Android)\n2. Open the app and enter your location or zip code to find nearby t